In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score

In [5]:
import string

def load_data():
    filename = "../data/twitter_data_fixed.pkl"
    print("Loading data from file: " + filename)
    data = pickle.load(open(filename, 'rb'))
    x_text = []
    labels = []
    ids = []
    for i in range(len(data)):
        text = "".join(l for l in data[i]['text'] if l not in string.punctuation)
        x_text.append((data[i]['text']).encode('utf-8'))
        labels.append(data[i]['label'])
    return x_text,labels

In [59]:
def get_model(name):
    if name == "lr":
        model = LogisticRegression()
    elif name == "svm":
        model = LinearSVC()
    elif name == "rf":
        model = RandomForestClassifier(n_jobs=-1)
    elif name == "nb":
        model = MultinomialNB()
    else:
        return None
    return model

def evaluate_model(modelname, features, labels):
    model = get_model(modelname)
    kfoldcv = KFold(n_splits = n_folds)
    scores = []
    n = 0
    for train_index, test_index in kfoldcv.split(features):
        n += 1
        print(n)
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        model.fit(X=X_train, y=y_train)
        y_pred = model.predict(X_test)
        f1 = f1_score(y_test, y_pred, average=None)
        scores.append(f1)

    return np.mean(scores, axis=0)

In [56]:
model = get_model("svm")
model.fit(X=features, y=labels)
labels_pred = model.predict(features)

In [57]:
f1 = f1_score(labels, labels_pred, average=None)

In [58]:
f1

array([0.89731627, 0.76802683, 0.69364383])

In [24]:
x_text, labels_og = load_data()

Loading data from file: ../data/twitter_data_fixed.pkl


In [25]:
labels, uniques = pd.factorize(labels_og)

In [26]:
comments = pd.DataFrame({'comment': x_text, 'attack': labels})
comments['comment'] = comments['comment'].str.decode("utf-8")
comments['comment'] = comments['comment'].str.replace('(\@\w+.*?)',"")

C:\Users\weetb\AppData\Local\Temp/ipykernel_15348/3401942055.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  comments['comment'] = comments['comment'].str.replace('(\@\w+.*?)',"")


In [27]:
comments

,comment,attack
0,rt : another bloody instant restaurant week ? ...,0
1,this video of the peshmerga decimating isis ...,0
2,oh really ? no more instant restaurants ? th...,0
3,rt : it has not been a good few weeks for #isi...,0
4,rt : i don’t need femisnsn because men carry h...,0
...,...,...
16085,"rt : ""i want equal rights , but i still want ...",2
16086,rt : go ahead and call me sexist but scandalou...,2
16087,"i have had the epic , but i always kept it p...",0
16088,so do you think that the daesh are planning t...,0


In [29]:
textdata = comments['comment']

In [30]:
comments.to_csv("../data/twitter_data_preprocessed.csv", index=False)

In [60]:
models = ["lr", "svm", "rf", "nb"]
representations = ["character n-grams", "word unigrams"]
n_folds = 10

results = []
for rep in representations:
    if rep == "word unigrams":
        transformer = CountVectorizer(analyzer="word", stop_words="english")
    else:
        transformer = CountVectorizer(analyzer = "char", ngram_range=(1,2))
    count_textdata = transformer.fit_transform(textdata)
    freq_transfomer = TfidfTransformer()
    freq_textdata = freq_transfomer.fit_transform(count_textdata)
    features = freq_textdata

    scores = []
    for modelname in models:
        print(rep, modelname)
        f1_scores = evaluate_model(modelname, features, labels)
        scores.append(f1_scores)

    results.append(pd.Series(scores, index=models))

df = pd.concat(results, axis=0, keys=representations)

character n-grams lr
1


C:\Users\weetb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


2


C:\Users\weetb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


3


...

9


C:\Users\weetb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


10


C:\Users\weetb\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


word unigrams svm
1
2
3
4
5
6
7
8
9
10
word unigrams rf
1
2
3
4
5
6
7
8
9
10
word unigrams nb
1
2
3
4
5
6
7
8
9
10


In [61]:
print(uniques)
print(df)

['none' 'racism' 'sexism']
character n-grams  lr     [0.8751335072171373, 0.685327978133294, 0.6180...
                   svm    [0.8782619140294108, 0.6970001698013153, 0.642...
                   rf     [0.8661223589261834, 0.534585081271872, 0.5346...
                   nb     [0.8189128812320003, 0.02636348677077856, 0.13...
word unigrams      lr     [0.8823246105420989, 0.6730464408470065, 0.646...
                   svm    [0.8871198073094947, 0.721519829764152, 0.6932...
                   rf     [0.8871867733014165, 0.7372396761164366, 0.665...
                   nb     [0.8480896742923367, 0.40042297496296236, 0.39...
dtype: object


In [64]:
df.name = "F1_Twitter"
df.to_frame().to_csv("results_twitter.csv", index_label=['processing', 'model', ['none', 'racism', 'sexism']])